# Install

# Import

In [1]:
import pandas as pd
pd.set_option('future.no_silent_downcasting', True)
import re,os,json
import logging
logging.getLogger('pdfminer.pdfinterp').setLevel(logging.ERROR)
logging.getLogger('pdfminer.layout').setLevel(logging.ERROR)

# fn

In [3]:
from modules.ITR1 import ITR1Sections,ITR1BatchProcessor

In [ ]:
# input_file_path = 'OUTPUT/ ITR 1-2 AY 2025-26.pdf'
# output_file_path = input_file_path.replace(".pdf", "_extracted.txt")
# config_path = "config/ITR1_header.json"
# itr = ITR1Sections(input_file_path, output_file_path, config_path)
# # itr

In [ ]:
# itr.sections

In [ ]:
# pd.DataFrame(itr.extracted[0:22])

In [ ]:
# print(itr.table_start_ptr)
# print(itr.ftr_row_map)
# print(itr.hdr_map)

In [ ]:
# itr.sections

In [ ]:
# itr.dataframes

In [ ]:
# sections = extract_sections(itr.extracted[0:30], itr.table_start_ptr, itr.ftr_row_map, itr.hdr_map)
# sections

In [ ]:
# assert False

# Process ONE PDF

In [ ]:
input_file_path = "OUTPUT/ITR1AY2023-24.pdf"
print(f"File exists: {os.path.exists(input_file_path)}")
output_file_path = input_file_path.replace(".pdf", "_extracted.txt")
config_path = "config/ITR1_header.json"

itr = ITR1Sections(input_file_path, output_file_path, config_path)

print("PAN:", itr.pan)
print("Ack:", itr.ack)
print("DoF:", itr.dof)

In [ ]:
# example: see one section
part_b_df = itr.get_section("PART_B")
part_b_df.head(1)

In [ ]:
viewer = itr.debug_viewer()
viewer

In [ ]:
# viewer.show()
viewer.by_section("PART_B")

# Process ALL PDFs

In [ ]:
input_dir = "OUTPUT"
config_path = "config/ITR1_header.json"
processor = ITR1BatchProcessor(input_dir, config_path)
processor.process_all()

In [ ]:
metadata_df = processor.metadata()
print("Metadata shape:", metadata_df.shape)
metadata_df.head(1)

In [ ]:
itr1_0=metadata_df.loc[0, "itr_obj"]
itr1_0

In [ ]:
processor.export_by_pan()              # uses OUTPUT as base
# or:
# processor.export_by_pan(output_dir="OUTPUT/combined")

# End